# Assignment 12

### 1.复习上课内容

### 2.回答以下理论问题

#### 1. 请写一下TF-IDF的计算公式

$$W_{t,d}=(1+\log_{10}{{tf}_{t,d}})\times(\log_{10}{N/df_t}) $$

#### 2. LDA算法的基本假设是什么？

Per-document topic distributions: The topics of each document obey a probability distribution.

Per-topic word distributions: The words of each topic obey  a probability distribution

#### 3. 在TextRank算法中构建图的权重是如何得到的？

权重通过迭代获得, 在初始状态时,step=0，$WS= 1$

every step 迭代$$ WS(V_i) = (1-d) + d*\sum_{j\in In(V_i)}\frac{w_{ji}}{\sum_{V_k \in Out(V_j)}w_{jk}}WS(V_j)$$

#### 4. 什么是命名实体识别？ 有什么应用场景？

Named entity recognition(NER) = to find each mention of a named entity in the text and label its type. 
NER types are application specific:
   Commonly include people, places, and organizations
   Biomedical domain: names of genes and proteins
   Education: codes of college courses

#### 5.NLP主要有哪几类任务 ？

Classification task  (sentiment classification)

Generation task (Machine translation)

Generation task (Question answering)

Text summarization


### 3.实践题

#### 3.1 手动实现TextRank算法 (在新闻数据中随机提取100条新闻训练词向量和做做法测试）

In [1]:
import gensim
import pandas as pd
import jieba
import re
#import stanfordcorenlp
import math
import numpy as np
import random
import tqdm

In [2]:
MAX_WINDOWS=5
data_source = "../Lecture_12/export_sql_1558435/sqlResult_1558435.csv"

In [5]:
df = pd.read_csv(data_source,encoding='gb18030')

In [7]:
content =df['content'][:100].tolist()

In [11]:
content =[ ''.join(token(w)) for w in content]

In [13]:
len(content)

100

In [12]:
content = [ cut(text) for text in content ]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\tb\AppData\Local\Temp\jieba.cache
Loading model cost 1.120 seconds.
Prefix dict has been built successfully.


In [10]:
def cut(text): return ' '.join(jieba.cut(text))

In [9]:
def token(string):
    return re.findall(r'[\w]+',string)

In [13]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec, word2vec, KeyedVectors

In [14]:
sentence = [s.split() for s in content]

In [29]:
sentence[0]

['此外',
 '自',
 '本周',
 '6',
 '月',
 '12',
 '日起',
 '除',
 '小米',
 '手机',
 '6',
 '等',
 '15',
 '款',
 '机型',
 '外',
 '其余',
 '机型',
 '已',
 '暂停',
 '更新',
 '发布',
 '含',
 '开发',
 '版',
 '体验版',
 '内测',
 '稳定版',
 '暂不受',
 '影响',
 '以',
 '确保',
 '工程师',
 '可以',
 '集中',
 '全部',
 '精力',
 '进行',
 '系统优化',
 '工作',
 '有人',
 '猜测',
 '这',
 '也',
 '是',
 '将',
 '精力',
 '主要',
 '用到',
 'MIUI9',
 '的',
 '研发',
 '之中',
 'MIUI8',
 '去年',
 '5',
 '月',
 '发布',
 '距今已有',
 '一年',
 '有余',
 '也',
 '是',
 '时候',
 '更新换代',
 '了',
 '当然',
 '关于',
 'MIUI9',
 '的',
 '确切',
 '信息',
 '我们',
 '还是',
 '等待',
 '官方消息']

In [15]:
path = get_tmpfile("all_vec.model")        # 创建临时文件
#sentences = word2vec.LineSentence(content)        # 读取预料

# Word2Vec的参数说明:https://blog.csdn.net/szlcw1/article/details/52751314
# size:特征向量的维度; window:当前词与预测词在一个句子中的最大距离；
# min_count: 词频少于min_count次数的单词会被丢弃掉
# workers:控制训练的并行数
model = Word2Vec(sentence, size=100, window=10, min_count=0, workers=4)        # 训练模型
model.save("all_vec.model")         # 保存模型
model.wv.save_word2vec_format('all_vec_format.txt')        # 将词向量以文本形式输出

In [16]:
print(model.wv['手机'])

[ 0.15399468  0.4039489   0.05288561 -0.07181797  0.10743837 -0.10339998
 -0.23051383 -0.0026197  -0.09466308  0.79761595  0.3082908   0.3225964
  0.48701683  0.32459816  0.145724   -0.21977541 -0.3929227  -0.62011665
  0.32026753 -0.51316273  0.7719842  -0.27778202 -0.09495185 -0.4440521
  0.09031882  0.68165106  0.34470513  0.3622547  -0.9335784   0.3944233
 -0.02168757 -0.3797212   0.01692033 -0.13177791  0.038302    0.36723334
 -0.43965077 -0.2896048  -0.67428327  0.16977854  0.13396578 -0.09495805
 -0.04414823  0.13978401 -0.539076   -0.05339715 -0.14162105 -0.153299
  0.4337643  -0.33594924  0.10048851 -0.4140766   0.35931322 -0.15829597
 -0.53423095 -0.40989023 -0.5097661   0.30464458 -0.51953924 -0.11486758
  0.67521834 -0.17281787  0.12620965  0.08769912  0.16522962 -0.25121883
  0.21232535 -0.27337074  0.1913451  -0.21566556 -0.4190616   0.17935924
 -0.20057602 -0.05942836  0.27050883 -0.11616302  0.30129746 -0.03749178
 -0.6100178   0.10928423  0.26082936  0.87557876  0.0760

In [17]:
from scipy.spatial.distance import cosine

In [18]:
def distance(x,y): return cosine(x,y)

In [19]:
distance(model.wv['手机'],model.wv['MIUI9'])

0.014828383922576904

In [20]:
class node:
    def __init__(self,string):
        self.vec = model.wv[string]
        self.children= []
        self.name=string
        self.weight=1

In [21]:
def setGraph(doc,maxsize=5):
    lsnode=[]
    dicnode={}
    for i,text in list(enumerate(doc))[:-maxsize]:
        if text not in dicnode:#如果没有节点，创建新节点。
            x=node(text)
            lsnode.append(x)
            dicnode[text]=x
        else:x=dicnode[text]#否则指向已经创建的节点
            
        for j in range(1,maxsize):
            if doc[i+j]==text:
                continue
            if doc[i+j] in dicnode:
                x1 = dicnode[doc[i+j]]
            else:
                x1 = node(doc[i+j])
                lsnode.append(x1)
                dicnode[doc[i+j]]=x1
                
            if x1 not in x.children:
                #simi=distance(x.vec,x1.vec)
                x.children.append(x1)
                x1.children.append(x)
                
    return dicnode,lsnode
        

In [48]:
from functools import lru_cache

In [52]:

def iteratble(lsnode,iteration=500):
    for i in tqdm(list(range(iteration))):
        for node in lsnode:
            node.weight = (1-0.85)+ 0.85*sum(child.weight*distance(child.vec,node.vec)/sum([distance(nodechild.vec,child.vec) for nodechild in child.children]) for child in node.children)

In [29]:
dic,ls=setGraph(sentence[0])

In [24]:
print('hello')

hello


In [66]:
lst=[[1,2,3],[4,5,6]]

In [71]:
for q in lst:
    print(q)

[1, 2, 3]
[4, 5, 6]


In [72]:
def ss(lst):
    s= sum( [i for i in q for q in lst])
    return s

In [73]:
ss(lst)

30

In [74]:
iteratble(ls)

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [04:20<00:00,  1.92it/s]


In [56]:
ls=sorted(ls,key=lambda x:x.weight,reverse=True)

In [57]:
l=[]
for s in ls:
    l.append((s.name,s.weight))

In [58]:
l

[('本周', 3.258852576467161),
 ('系统优化', 2.617370357239604),
 ('有人', 2.5925704959207865),
 ('距今已有', 2.3945381965906294),
 ('确切', 2.246343657586842),
 ('有余', 2.2035580324395716),
 ('更新换代', 1.9744231660841356),
 ('体验版', 1.9065130337323257),
 ('精力', 1.758911868483012),
 ('内测', 1.7438040928624745),
 ('MIUI9', 1.7319974392513782),
 ('用到', 1.6782446528402541),
 ('稳定版', 1.6374812735787247),
 ('暂不受', 1.6043439823084986),
 ('MIUI8', 1.5682868612199754),
 ('月', 1.4644639113860194),
 ('其余', 1.3615336305314156),
 ('之中', 1.336709940662061),
 ('是', 1.2521120170532232),
 ('也', 1.0967296587552857),
 ('猜测', 1.0900026046125308),
 ('发布', 1.044183161098673),
 ('机型', 1.0085783288420023),
 ('除', 0.9535130125864043),
 ('6', 0.8924539686660256),
 ('的', 0.8875342302489816),
 ('日起', 0.8688931363421769),
 ('版', 0.8023154487920857),
 ('12', 0.7917722057278502),
 ('含', 0.764570186159978),
 ('进行', 0.739791813587607),
 ('工作', 0.7314256693506848),
 ('这', 0.7294934685549358),
 ('影响', 0.7045565873757405),
 ('小米', 0.688341

In [32]:
from tqdm import tqdm

In [35]:
for i in tqdm(range(1000)):
    pass

100%|██████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 333543.06it/s]


 提示：
 1. 确定窗口，建立图链接。   
 2. 通过词向量相似度确定图上边的权重
 3. 根据公式实现算法迭代(d=0.85)

#### 选做 1.  提取新闻人物里的对话。(使用以上提取小数据即可）

提示：    
1.寻找预料里具有表示说的意思。    
2.使用语法分析提取句子结构。    
3.检测谓语是否有表示说的意思。

#### 选择2. ： 电影评论分类。

在这个作业中你要完成一个电影评论分类任务。

1.数据获取。（采用爬虫技术爬取相关网页上的电影评论数据，例如猫眼电影评论，豆瓣电影评论）

2.把所获得数据分解为训练集，验证集和测试集。

3.选用相应算法构建模型，并测试。

In [75]:
df_movie =pd.read_csv('./movie_comments.csv')

D:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [76]:
df_movie.head()

,id,link,name,comment,star
0,1,https://movie.douban.com/subject/26363254/,战狼2,吴京意淫到了脑残的地步，看了恶心想吐,1
1,2,https://movie.douban.com/subject/26363254/,战狼2,首映礼看的。太恐怖了这个电影，不讲道理的，完全就是吴京在实现他这个小粉红的英雄梦。各种装备轮...,2
2,3,https://movie.douban.com/subject/26363254/,战狼2,吴京的炒作水平不输冯小刚，但小刚至少不会用主旋律来炒作…吴京让人看了不舒服，为了主旋律而主旋...,2
3,4,https://movie.douban.com/subject/26363254/,战狼2,凭良心说，好看到不像《战狼1》的续集，完虐《湄公河行动》。,4
4,5,https://movie.douban.com/subject/26363254/,战狼2,中二得很,1


In [78]:
comment =df_movie['comment',].tolist()

In [79]:
stop_words = []
with open('../baidu_stopwords.txt',encoding='utf-8') as f:
    for word in f.readlines():
        stop_words.append(word.strip())

In [80]:
len(comment)

261497

In [83]:
from gensim import corpora, models
import jieba.posseg as jp,jieba

In [85]:
flags = ('n','nr', 'ns', 'nt', 'eng', 'v', 'd')

In [88]:
from tqdm import tqdm

In [89]:
words_ls = []
for text in tqdm(comment):
    words = [w.word for w in jp.cut(str(text)) if w.flag in flags and w.word not in stop_words]
    words_ls.append(words)

100%|█████████████████████████████████████████████████████████████████████████| 261497/261497 [25:16<00:00, 172.48it/s]


In [90]:
dictionary = corpora.Dictionary(words_ls)

In [91]:
corpus = [dictionary.doc2bow(words) for words in words_ls] 

In [92]:
lda = models.ldamodel.LdaModel(corpus=corpus,id2word=dictionary,num_topics=5)

In [93]:
lda.save('model_lda')

In [94]:
for topic in lda.print_topics(num_words=10):
    print(topic)

(0, '0.024*"爱" + 0.019*"人" + 0.011*"都" + 0.009*"最" + 0.008*"可爱" + 0.008*"世界" + 0.006*"孩子" + 0.006*"做" + 0.006*"爱情" + 0.006*"不"')
(1, '0.028*"人" + 0.025*"很" + 0.019*"都" + 0.015*"电影" + 0.011*"故事" + 0.011*"却" + 0.010*"不" + 0.009*"音乐" + 0.008*"镜头" + 0.007*"影片"')
(2, '0.031*"电影" + 0.019*"还" + 0.018*"故事" + 0.016*"太" + 0.016*"拍" + 0.014*"都" + 0.013*"剧情" + 0.013*"导演" + 0.013*"很" + 0.010*"不"')
(3, '0.069*"看" + 0.028*"都" + 0.023*"不" + 0.022*"说" + 0.021*"电影" + 0.019*"想" + 0.014*"没" + 0.013*"去" + 0.011*"片子" + 0.010*"还"')
(4, '0.059*"喜欢" + 0.033*"不" + 0.024*"好看" + 0.015*"青春" + 0.012*"最" + 0.012*"太" + 0.010*"电影" + 0.010*"男人" + 0.009*"女人" + 0.008*"懂"')


In [95]:
text= comment[10]

In [96]:
text

'犯我中华者虽远必诛，是有多无脑才信这句话。'

In [97]:
words = [[w.word for w in jp.cut(text) if w.flag in flags and w.word not in stop_words]]

In [98]:
text_corpus = [dictionary.doc2bow(word) for word in words]

In [99]:
list(lda[text_corpus])

[[(0, 0.032633934),
  (1, 0.032524526),
  (2, 0.38514617),
  (3, 0.51721495),
  (4, 0.0324804)]]

#### 选择3：文章自动续写

在这个作业中你要完成一个文章自动续写的模型。

1.数据获取。（根据你的兴趣采用爬虫技术爬去相关网站上的文本数据内容：比如故事网站，小说网站等）

2.选取模型，并训练。

3.展示一些你模型的输出例子。